In [2]:
import xml.etree.ElementTree as ET
import re
tree = ET.parse('data.xml')
root = tree.getroot()

In [3]:

citylist =dict(Berlin="BERLIN IS AWESOME")
STOPS = ["\{", "\}", "\[", "\]", "\\n","\\"]

counter = 0
key = ""
description = ""

word = 'fubar'
re_title = re.compile(r'title')
re_revision = re.compile(r'revision')
re_text = re.compile(r'text')
re_cutter = re.compile(r'[^=]*==')


for child in root:
    if (child.tag == "{http://www.mediawiki.org/xml/export-0.10/}siteinfo"):
        root.remove(child)
    else:
        #print 'CHILD'
        #print child.tag, child.attrib
        for subchild in child:
            if re_title.search(subchild.tag):
                #print "TITLEEEE"
                #print subchild.text
                key = subchild.text
            if re_revision.search(subchild.tag):
               # print 'REVISION', subchild.tag, subchild.attrib
                for text in subchild.findall('{http://www.mediawiki.org/xml/export-0.10/}text'):
                    if(len(text.text)>15):
                        description = text.text
                        for section in re_cutter.finditer(description):# Remove anything after first == appearance
                            description = section.group(0)
                            description = description[:-3]#Remove also the == that were included in the regex
                            break;
                        description = re.sub("\[\[[^\]]*\]", " ",description) # Remove everything between [[...]]
                        description = re.sub("\{\{[^\}]*\}", "", description) # Remove everything between {{...}}
                        description = re.sub("'''", "'", description) # Handle the ''''
                        description = re.sub("[\\\{\}\[\]\\n]","", description) # Remove stuff

                    
            citylist[key] = description
                    

        counter += 1
        if (counter >= 100):
            break;
    
    

In [64]:
#citylist.viewkeys()

In [4]:
abr = citylist["Abruzzo"]
abr

u" 'Abruzzo' is one of the twenty Italian regions. It is situated in the central part of  , to the east of Rome on the Adriatic Sea. Abruzzo's western border lies less than 50 miles due east of Rome. Abruzzo borders the region of Marche to the north, Lazio to the west and south-west, Molise to the south-east, and the Adriatic Sea to the east. Although geographically more of a central than southern region, ISTAT (the Italian statistical authority) considers it part of Southern Italy, a vestige of Abruzzo's historic association with the Kingdom of the Two Sicilies.Abruzzo is one of the most sparsely populated regions on the Italian peninsula. Always a wild and empty region, since the Second World War it has depopulated further, as people have left the land, and the traditional mainstay of sheep farming, for the cities and for America. A vacation in Abruzzo offers a taste of the unspoiled Italy."

# Seq2Vec own Model with Word2Vec average Vector

In [1]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import cPickle, gzip
import numpy as np
import pandas as pd


In [2]:
EMBEDDING_FILE = './model/GoogleNews-vectors-negative300.bin'
#model = Word2Vec.load(EMBEDDING_FILE)

word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, \
        binary=True)
print "Loaded model"

Loaded model


In [146]:
#Load Data
df = pd.read_csv('./proc_data.csv')
descriptions = np.array(df.loc[:]['description'])
descriptions_vec = [vectorize(word2vec, description) for description in descriptions]

In [148]:
def vectorize(word2vec, sentence, num_features=300):
    mywords = sentence.split(" ")
    myvector = np.zeros((num_features),dtype="float32")

    i = 0
    for word in mywords:
        #print word
        if word in word2vec.vocab:
            myvector = np.add(myvector,word2vec[word]) # Adding every new vector
            i+=1
    featureVec = np.divide(myvector, i) # and in the end dividing it by the number of words

    return featureVec

def eucDistance(vector1, vector2):
    euclid = 0.0
    for (dim1, dim2) in zip(vector1, vector2):
        euclid = euclid +(dim1-dim2)*(dim1-dim2)
        #print dim1, dim2
    euclid = np.sqrt(euclid)
    return euclid
def maxDistance(vector1, vector2):
    return max(np.abs(np.substract(vector1,vector2)))
def minDistance(vec1, vec2):
    return min(np.abs(np.substract(vec1,vec2)))
def findBestMatch(vec1, vecarray):
    distances = np.zeros(len(vecarray))
    for i in range(0,len(vecarray)):
        distances[i] = eucDistance(vec1, vecarray[i])
    index = distances.argmin()
    return index             
        
def dfToVecArray(word2vec, df):
    vecarray = 1 # NNEDS TO BE DEFINED
    return vecarray

In [152]:
bsp1 = "Hi I am rather cool"
bsp2 = "Yo where have you been"
myvec1 = vectorize(word2vec, bsp1)
myvec2 = vectorize(word2vec, bsp2)
vecarray = [myvec1, myvec2]

#euc = eucDistance(myvec1, myvec2)
descriptions[findBestMatch(myvec1,vecarray)]

' shertogenbosch  commonly known  den bosch  city south netherlands capital province north brabant stronghold vital protection young dutch nation den bosch charming wellpreserved medieval centre wander winding streets see saint johns cathedral pick street terrace market square relax chilling beer take boat see part unique binnendieze subterranean network canals city head south part town ancient ramparts still mark border city beginning natural reserve area short den bosch lacks fame makes charm place well worth visiting'